# Dataset Generation Notebook
This notebook contains the code required to generate the challenge datasets from the raw sources. Checkout the accompanying paper for sources and links to said raw datasets.

In [99]:
#Import
import os
import pandas as pd
import numpy as np
from collections import defaultdict

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

____
## 1. Ingest and Parse Raw Data

#### Main Yield Data

In [100]:
#Read data
pth = 'data/Production_Crops_Livestock_E_All_Data/Production_Crops_Livestock_E_All_Data_NOFLAG.csv'
df1 = pd.read_csv(pth,encoding='latin-1')

In [101]:
#Selected
items_sl = ['Cereals',
            'Maize',
            'Sugar Crops Primary',
            'Potatoes',
            'Rice',
            'Rice, paddy (rice milled equivalent)',
            'Beans, dry',
            'Wheat',
            'Sorghum',
            'Barley',
            'Millet',
            'Oats',
            'Rye',
            'Buckwheat',
            'Molasses',
           ]

In [102]:
#Take stock of data we have on collected categories
df1 = df1.loc[df1.Item.isin(items_sl)]

In [103]:
#Count how many years of data are available and countries
country_ct = df1.Area.value_counts()
year_cols = df1.columns[9:]

non_na_entries = (~df1[year_cols].isna())
non_na_entries.sum().sum() #255030 datapoints

255030

#### Food Balance

In [104]:
#Food balance dataset
pth = 'data/FoodBalanceSheets_E_All_Data/FoodBalanceSheets_E_All_Data_NOFLAG.csv'
df_foodbal = pd.read_csv(pth,encoding='latin-1')

In [105]:
df_foodbal.Element.value_counts()

#Use Food, Feed, Losses
#For info of def check out https://www.fao.org/faostat/en/#data/FBS

Domestic supply quantity                  25113
Import Quantity                           24640
Protein supply quantity (g/capita/day)    23857
Fat supply quantity (g/capita/day)        23852
Food supply (kcal/capita/day)             23833
Food                                      23402
Food supply quantity (kg/capita/yr)       23305
Export Quantity                           23043
Residuals                                 22126
Stock Variation                           21214
Production                                19162
Losses                                    12772
Other uses (non-food)                     12714
Processing                                11326
Feed                                      11034
Tourist consumption                       10911
Seed                                       7154
Total Population - Both sexes               214
Name: Element, dtype: int64

In [106]:
df2 = df_foodbal.loc[df_foodbal.Element.isin(['Food','Feed','Losses'])]
# df2.head(5) #include a lot of other non selected item elements, to filter or not to filter?

In [107]:
#Aggregate
df2_feed = df2.loc[df2.Element == 'Feed'].groupby(['Area Code']).sum()
df2_feed.drop(['Item Code','Element Code'],axis=1,inplace=True)
df2_feed['units'] = '1000 tonnes'

In [108]:
#Aggregate
df2_losses = df2.loc[df2.Element == 'Losses'].groupby(['Area Code']).sum()
df2_losses.drop(['Item Code','Element Code'],axis=1,inplace=True)
df2_losses['units'] = '1000 tonnes'

In [109]:
#Aggregate
df2_Food = df2.loc[df2.Element == 'Food'].groupby(['Area Code']).sum()
df2_Food.drop(['Item Code','Element Code'],axis=1,inplace=True)
df2_Food['units'] = '1000 tonnes'

#### Macro Data

In [110]:
#Try Macro data
pth = 'data/Macro-Statistics_Key_Indicators_E_All_Data/Macro-Statistics_Key_Indicators_E_All_Data_NOFLAG.csv'
df_macro = pd.read_csv(pth,encoding='latin-1')


In [111]:
df_macro.Item.value_counts()

#select
macro_it_se = ['Gross Domestic Product',
               'Gross Output (Agriculture)']

In [112]:
df_macro.Element.value_counts()

#select
macro_el_se = ['Value US$']

In [113]:
df3_gdp = df_macro.loc[(df_macro.Item.isin(['Gross Domestic Product'])) & (df_macro.Element.isin(macro_el_se))]
df3_agi_out = df_macro.loc[(df_macro.Item.isin(['Gross Output (Agriculture)'])) & (df_macro.Element.isin(macro_el_se))]

In [114]:
#Aggregate
df3_gdp = df3_gdp.groupby(['Area Code']).sum() #can skip Element agg here, just 1 value
df3_gdp.drop(['Item Code','Element Code'],axis=1,inplace=True)
df3_gdp['units'] = 'millions $'

In [115]:
df3_agi_out = df3_agi_out.groupby(['Area Code']).sum() #can skip Element agg here, just 1 value
df3_agi_out.drop(['Item Code','Element Code'],axis=1,inplace=True)
df3_agi_out['units'] = 'millions $'

#### Inputs_Pesticides_Use_E_All_Data

In [116]:
#Try Macro data
pth = 'data/Inputs_Pesticides_Use_E_All_Data/Inputs_Pesticides_Use_E_All_Data_NOFLAG.csv'
df_pest = pd.read_csv(pth,encoding='latin-1')


In [117]:
df4 = df_pest.loc[(df_pest.Item.isin(['Pesticides (total)']))]

In [118]:
df4.shape

(207, 39)

In [119]:
#Aggregate
df4 = df4.groupby(['Area Code']).sum() #can skip Element agg here, just 1 value
df4.drop(['Item Code','Element Code'],axis=1,inplace=True)
df4['units'] = 'tonnes'

#### Fire

In [120]:
pth = 'data/Emissions_Land_Use_Fires_E_All_Data/Emissions_Land_Use_Fires_E_All_Data_NOFLAG.csv'
df_fire = pd.read_csv(pth,encoding='latin-1')


In [121]:
df5 = df_fire.loc[df_fire.Element == 'Burned Area']

In [122]:
#Aggregate
df5 = df5.groupby(['Area Code']).sum() #can skip Element agg here, just 1 value
df5.drop(['Item Code','Element Code'],axis=1,inplace=True)
df5['units'] = 'ha'

#### abcnews-date-text


In [123]:
pth = 'data/abcnews-date-text.csv'
df_news = pd.read_csv(pth)


In [124]:
print(df_news.shape)

df_news.head(10)

(1244184, 2)


,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers
5,20030219,ambitious olsson wins triple jump
6,20030219,antic delighted with record breaking barca
7,20030219,aussie qualifier stosur wastes four memphis match
8,20030219,aust addresses un security council over iraq
9,20030219,australia is locked into war timetable opp


In [125]:
farms = []

for i in df_news.headline_text.values:
    if 'farm' in i or 'agri' in i:
        farms.append(i)

In [126]:
len(farms)

17308

In [127]:
print(farms[:10])

['plan to encourage farmers into plantation timber', 'groups praise outgoing opposition agriculture', 'rain boosts farmers confidence', 'council approves poultry farm', 'farmers celebrate good rain', 'group gets clearer understanding of farmers gm', 'vic govt gets farmers drought update', 'victorian farmers hope rain will break drought', 'rain brings relief for act farmers', 'drought assistance will continue for farmers carr']


___
## 2. Merge Datasets

In [128]:
df1.index = df1['Area Code']
df1.drop(['Area Code','Area Code (M49)'],axis=1,inplace=True)

In [129]:
#Fine tune df1
df1.Element.value_counts() #Let's concentrate on production?

df1_ = df1.loc[df1.Element == 'Production']

In [130]:
#Merge! 
col1 = [i + '_AgriProd' if 'Y' in i else i for i in df1_.columns ]
col2 = [i + '_AnnualFeedProd' for i in df2_feed.columns]

df1_.columns = col1
df2_feed.columns = col2 

In [131]:
df = df1_.merge(df2_feed,how='left',right_index=True,left_index=True)

In [132]:
#Merge! 
col2 = [i + '_AnnualFoodProd' for i in df2_Food.columns]

df2_Food.columns = col2 

In [133]:
df = df.merge(df2_Food,how='left',right_index=True,left_index=True)

In [134]:
#Merge! 
col2 = [i + '_AnnualAgriLosses' for i in df2_losses.columns]

df2_losses.columns = col2 

In [135]:
df = df.merge(df2_losses,how='left',right_index=True,left_index=True)

In [136]:
#Merge! 
col2 = [i + '_GDP' for i in df3_gdp.columns]

df3_gdp.columns = col2 

In [137]:
df = df.merge(df3_gdp,how='left',right_index=True,left_index=True)

In [138]:
#Merge! 
col2 = [i + '_GrossAgriVal' for i in df3_agi_out.columns]

df3_agi_out.columns = col2 

In [139]:
df = df.merge(df3_agi_out,how='left',right_index=True,left_index=True)

In [140]:
#Merge! 
col2 = [i + '_Pesticide' for i in df4.columns]

df4.columns = col2 

In [141]:
df = df.merge(df4,how='left',right_index=True,left_index=True)

In [142]:
#Merge! 
df5.drop('Source Code',axis=1,inplace=True)

col2 = [i + '_Fire' for i in df5.columns]

df5.columns = col2 

In [143]:
df = df.merge(df5,how='left',right_index=True,left_index=True)

In [144]:
df.shape

(2005, 269)

##### Done Merging!

In [145]:
df_news #this is unmerged, leave to the sideas a separate dataset and handled in feature engineering

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers
...,...,...
1244179,20211231,two aged care residents die as state records 2...
1244180,20211231,victoria records 5;919 new cases and seven deaths
1244181,20211231,wa delays adopting new close contact definition
1244182,20211231,western ringtail possums found badly dehydrate...


___
## 3. Generate Challenge Dataset

#### Break into Challenges
1. Challenge 1 
    - Try to predict next year's production (in tonnes) for crop (item) X, given last 20 years of data WIHOUT AGRI_PROD feature.
2. Challenge 2
    - Try to predict next year's production (in tonnes) for crop (item) X, given last 5 year of data.
2. Challenge 3
    - Try to predict next year's production (in tonnes) for crop (item) X, given last 1 year of data.

#### Challenge 1

**Note**: Min yearis 1961, max year is 2020 on yield, let's base it off of this

In [146]:
keys = ['_AgriProd',
        '_AnnualFeedProd',
        '_AnnualFoodProd',
        '_AnnualAgriLosses',
        '_GDP',
        '_GrossAgriVal',
        '_Pesticide',
        '_Fire']

#NEWS DATA NOT INCLUDED HERE!

In [147]:
n = 21
combs_ = np.arange(1961,2021,1)
comb20 = []

for i in range(n,len(combs_)):
    comb20.append(combs_[i-n:i])

In [148]:
#Assemble Challenge 1 dataset
c1_dt = defaultdict(dict)
ct = 0

for idx,(k,v) in enumerate(df.iterrows()):
    
    if idx % 100 == 0:
        print("Done {}/{}...".format(idx,len(df)))
    
    area = v.Area
    areaCode = k
    item = v.Item
    prod_unit = v.Unit
    
    for j in comb20:
        
        c1_dt[ct] = {'area':area,
                      'areaCode':areaCode,
                      'item':item,
                      'prod_unit':prod_unit}
        
        for ijx,i in enumerate(j):

            for y in keys:
                
                try:
                    name = "Y{}{}".format(i,y)
                    val = v[name]
                    feat_name = 'y{}_{}'.format(ijx,y)
                    
                    c1_dt[ct][feat_name] = val
                    
                    
                except:
                    feat_name = 'y{}_{}'.format(ijx,y)
                    c1_dt[ct][feat_name] = np.nan
#                     print('Error!')
#                     print(name in v)

        ct+= 1

Done 0/2005...
Done 100/2005...
Done 200/2005...
Done 300/2005...
Done 400/2005...
Done 500/2005...
Done 600/2005...
Done 700/2005...
Done 800/2005...
Done 900/2005...
Done 1000/2005...
Done 1100/2005...
Done 1200/2005...
Done 1300/2005...
Done 1400/2005...
Done 1500/2005...
Done 1600/2005...
Done 1700/2005...
Done 1800/2005...
Done 1900/2005...
Done 2000/2005...


In [149]:
c1_df = pd.DataFrame.from_dict(c1_dt,orient='index')

In [150]:
c1_df.dropna(how='all',subset=c1_df.columns[4:],inplace=True) #drop all NA cols

In [151]:
c1_df.shape

(78077, 172)

#### Challenge 2

In [152]:
n = 6
comb5 = []

for i in range(n,len(combs_)):
    comb5.append(combs_[i-n:i])

In [153]:
#Assemble Challenge 1 dataset
c2_dt = defaultdict(dict)
ct = 0

for idx,(k,v) in enumerate(df.iterrows()):
    
    if idx % 100 == 0:
        print("Done {}/{}...".format(idx,len(df)))
    
    area = v.Area
    areaCode = k
    item = v.Item
    prod_unit = v.Unit
    
    for j in comb5:
        
        c2_dt[ct] = {'area':area,
                      'areaCode':areaCode,
                      'item':item,
                      'prod_unit':prod_unit}
        
        for ijx,i in enumerate(j):

            for y in keys:
                
                try:
                    name = "Y{}{}".format(i,y)
                    val = v[name]
                    feat_name = 'y{}_{}'.format(ijx,y)
                    
                    c2_dt[ct][feat_name] = val
                    
                    
                except:
                    feat_name = 'y{}_{}'.format(ijx,y)
                    c2_dt[ct][feat_name] = np.nan

        ct+= 1

Done 0/2005...
Done 100/2005...
Done 200/2005...
Done 300/2005...
Done 400/2005...
Done 500/2005...
Done 600/2005...
Done 700/2005...
Done 800/2005...
Done 900/2005...
Done 1000/2005...
Done 1100/2005...
Done 1200/2005...
Done 1300/2005...
Done 1400/2005...
Done 1500/2005...
Done 1600/2005...
Done 1700/2005...
Done 1800/2005...
Done 1900/2005...
Done 2000/2005...


In [154]:
c2_df = pd.DataFrame.from_dict(c2_dt,orient='index')

In [155]:
c2_df.dropna(how='all',subset=c2_df.columns[4:],inplace=True) #drop all NA cols

In [156]:
c2_df.shape

(105873, 52)

#### Challenge 3

In [157]:
n = 2
comb1 = []

for i in range(n,len(combs_)):
    comb1.append(combs_[i-n:i])

In [158]:
#Assemble Challenge 1 dataset
c3_dt = defaultdict(dict)
ct = 0

for idx,(k,v) in enumerate(df.iterrows()):
    
    if idx % 100 == 0:
        print("Done {}/{}...".format(idx,len(df)))
    
    area = v.Area
    areaCode = k
    item = v.Item
    prod_unit = v.Unit
    
    for j in comb1:
        
        c3_dt[ct] = {'area':area,
                      'areaCode':areaCode,
                      'item':item,
                      'prod_unit':prod_unit}
        
        for ijx,i in enumerate(j):

            for y in keys:
                
                try:
                    name = "Y{}{}".format(i,y)
                    val = v[name]
                    feat_name = 'y{}_{}'.format(ijx,y)
                    
                    c3_dt[ct][feat_name] = val
                    
                    
                except:
                    feat_name = 'y{}_{}'.format(ijx,y)
                    c3_dt[ct][feat_name] = np.nan

        ct+= 1

Done 0/2005...
Done 100/2005...
Done 200/2005...
Done 300/2005...
Done 400/2005...
Done 500/2005...
Done 600/2005...
Done 700/2005...
Done 800/2005...
Done 900/2005...
Done 1000/2005...
Done 1100/2005...
Done 1200/2005...
Done 1300/2005...
Done 1400/2005...
Done 1500/2005...
Done 1600/2005...
Done 1700/2005...
Done 1800/2005...
Done 1900/2005...
Done 2000/2005...


In [159]:
c3_df = pd.DataFrame.from_dict(c3_dt,orient='index')

In [160]:
c3_df.dropna(how='all',subset=c3_df.columns[4:],inplace=True) #drop all NA cols

In [91]:
c3_df.shape

(111696, 20)

In [92]:
#Save Out
c1_df.to_json('data/CROP_dataset/new_challenge1_dataset.json')
c2_df.to_json('data/CROP_dataset/new_challenge2_dataset.json')
c3_df.to_json('data/CROP_dataset/new_challenge3_dataset.json')

____
## End